In [36]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob 

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [37]:
DATA_DIR = '/kaggle/input'

# Directory to save logs and trained model
ROOT_DIR = '/kaggle/working'

# Data

In [4]:
arr=np.load(DATA_DIR+'/slacs-alike-lenses/SLACS_alike_lenses_xy64.npz')
data_images=arr['images']
data_masks=arr['masks']

In [5]:
print(data_images.shape)
print(data_masks.shape)

In [9]:
i=11

fig,ax=plt.subplots(1,3,figsize=(20,7))
ax[0].imshow(data_images[i,:,:,0])
ax[1].imshow(data_masks[i,:,:,0])
ax[2].imshow(data_masks[i,:,:,1])
for i in range(3):
    ax[i].axis('off')
plt.show()

In [10]:
from sklearn.model_selection import train_test_split

# Do train/test/val split for indices rather then for the actual arrays

#Extract test and val are 0.2 of dataset each
x_train,x_test,y_train,y_test= train_test_split(data_images, data_masks, test_size=0.4,random_state=42)
x_test, x_val , y_test, y_val= train_test_split(x_test, y_test, test_size=0.5,random_state=42)

# Mask-RCNN setup

In [11]:
!git clone 'https://github.com/leekunhee/Mask_RCNN'

In [38]:
# Import Mask RCNN
sys.path.append(os.path.join(ROOT_DIR, 'Mask_RCNN')) 

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs_SLACS")

weights_dir=DATA_DIR+'/mask-rcnn-coco-weights/mask_rcnn_coco.h5'
COCO_WEIGHTS_PATH = weights_dir

sys.path.append(ROOT_DIR)

from mrcnn.config import Config
from mrcnn import model as modellib,utils

import skimage

In [44]:
class CustomConfig(Config):
    """Configuration for training pneumonia detection on the RSNA pneumonia dataset.
    Overrides values in the base Config class.
    """
    
    # Give the configuration a recognizable name  
    NAME = 'GravLens'
    
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8 
    
    BACKBONE = 'resnet50'
    
    NUM_CLASSES = 3  # background + source + lens
    
    IMAGE_MIN_DIM = 64
    IMAGE_MAX_DIM = 64
    
    IMAGE_RESIZE_MODE = "square"
    IMAGE_CHANNEL_COUNT = 3
    
    USE_MINI_MASK = False
    MASK_SHAPE=[64,64]
    
    RPN_ANCHOR_SCALES = (32, 64, 128, 256)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 2
    DETECTION_MAX_INSTANCES = 2 # source and lens
    DETECTION_MIN_CONFIDENCE = 0.68 # 1sigma confidence
    DETECTION_NMS_THRESHOLD = 0.1

    STEPS_PER_EPOCH = 100
    
config = CustomConfig()
config.display()

In [45]:
#Config??

In [51]:
from tqdm import tqdm
############################################################
#  Dataset
############################################################


# TODO: Rewrite the methods to work with images from RAM, rather than from the disk 
class CustomDataset(utils.Dataset):

    def __init__(self, image_height,image_width, num_classes, class_map=None):
      # Introduce fields for images and masks. Handle train/test/val split using indices
      
        self._image_ids = []
        self.image_info = []

        self.image_height=image_height
        self.image_width=image_width

        # Grayscale images
        self.images=np.zeros((0,image_height,image_width,3))
        self.masks=np.zeros((0,image_height,image_width,num_classes))

        # Background is always the first class
        self.class_info = [{"source": "", "id": 0, "name": "BG"}]
        self.source_class_ids = {}

    def add_image(self, source, image_id, image, mask, **kwargs):
        image_info = {
            "id": image_id,
            "source": source
        }
        image_info.update(kwargs)
        self.image_info.append(image_info)
        
        # If grayscale. Convert to RGB for consistency.
        if image.shape[-1] != 3:
            image = skimage.color.gray2rgb(image[:,:,0])
        self.images=np.append(self.images,[image],axis=0)
        self.masks=np.append(self.masks,[mask],axis=0)
      

    def load_custom(self, dataset_images,dataset_masks):
        """Load the train/test/val datasets
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes
        self.add_class("object", 1, "source_light")
        self.add_class("object", 2, "lens_light")

        height,width=self.images.shape[-2:]

        for i,image in tqdm(enumerate(dataset_images)):
            self.add_image(
                source="object",  ## for a single class just add the name here
                image_id=i,  # use file name as a unique image id
                image=image,
                mask=dataset_masks[i])
            
    def load_image(self, image_id):
        return self.images[image_id]
    
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a beagle dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask_image = self.masks[image_id]

        # array (num_classes) with 1 for detection of class and 0 otherwise
        is_detection=(mask_image>0).any(axis=(0,1))

        # Ids of object start from 1 due to background
        detected_ids=1+np.where(is_detection)[0]

        mask = np.zeros([self.image_height, self.image_width, len(detected_ids)],
                        dtype=np.uint8)

        for i,id in enumerate(detected_ids):
            mask[:,:,i]=mask_image[:,:,id-1]

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), detected_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            raise NotImplementedError
        else:
            super(self.__class__, self).image_reference(image_id)

# Train

In [ ]:
# Training dataset.
dataset_train = CustomDataset(x_train.shape[1],x_train.shape[2],y_train.shape[-1])
dataset_train.load_custom(x_train, y_train)
dataset_train.prepare()

In [ ]:
# Validation dataset
dataset_val = CustomDataset(x_train.shape[1],x_train.shape[2],y_train.shape[-1])
dataset_val.load_custom(x_val, y_val)
dataset_val.prepare()

In [ ]:
def train(model,config,dataset_train,dataset_val):
    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=100,
                layers='heads')

In [ ]:
config = CustomConfig()
config.display()

print("Initialize model")
model = modellib.MaskRCNN(mode="training", config=config,model_dir=DEFAULT_LOGS_DIR)

# Load weights
weights_path = COCO_WEIGHTS_PATH
print("Loading weights ", weights_path)
# Exclude the last layers because they require a matching
# number of classes
model.load_weights(weights_path, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])

In [ ]:
print('Train model')
train(model,config,dataset_train,dataset_val)